In [412]:
import cvxpy as cp
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection  import train_test_split
import matplotlib.pyplot as plt

In [413]:
import numpy as np
from tqdm import tqdm
# from sklearn.metrics.pairwise import rbf_kernel
class SVMCust:
    def __init__(self) -> None:
        theta=1
        self.W=0
        self.b=0
        self.a=0
        self.C=0
        self.m=0
        self.f=0
    
    def kg(self,a, b, sigma=1):
        return np.exp((-(np.linalg.norm(a-b)**2))/(2*sigma**2))
        # print("dot:",np.dot(a,b))
        # # val=np.linalg.det(np.dot(a,b))
        # a=a.reshape((self.f,1))
        # b=b.reshape((self.f,1))
        # val=np.matmul(a.T,b)
        # print("as",a.shape, "b:",b.shape)
        # print(val.shape)
        return val
    # generating kernal matrix
    def k_mat(self,x, k_func=kg):
        m = x.shape[0]
        mat = np.zeros((m, m))
        for i in tqdm(range(m)):
            for j in range(m):
                mat[i, j] = self.kg(x[i], x[j])
        return mat

    def getAlphas(self):
        return self.a
    def getWeight(self):
        return self.W
    def getBias(self):
        return self.b
    def fit(self,X_train,y_train):
        print("shape:",X_train.shape)
        y_train[y_train==0]=-1
        (m,f)=X_train.shape
        self.f=f
        y_train=np.asarray(y_train)
        self.m=m
        X=np.array(X_train)
        # k = rbf_kernel(X)
        k=self.k_mat(X)
        print(np.all(np.abs(k-k.T) < 1e-08))
        # print(k)
        G=np.zeros((m,m))
        print("shaope:",G.shape)
        print("m:",m)
        for i in tqdm(range(0,m)):
            # for j in range(0,i+1):
                # print("i:",i,"j:",j)
                # print("s:",(y[i]*(np.inner(X[i],np.transpose(X[j])))*y[j]).shape)
                G[i]=(y_train[i]*np.multiply(y_train,(k[i])))
                # G[j][i]=G[i][j]
        # G=y*k*np.transpose(y)
        print(G)

        self.a = cp.Variable(m)
        self.C = cp.Parameter(nonneg=True)
        self.C.value = 0.02
        print("min eigne val:",np.linalg.eigvals(G).min())
        # G = cp.atoms.affine.wraps.psd_wrap(G)
        obj = cp.Minimize( (0.5) * cp.quad_form(self.a,G)-np.ones((1,m))@ self.a)
        constraints = [0<=self.a,self.a<=self.C ,np.transpose(y_train)@self.a==0]
        prob = cp.Problem(obj, constraints)
        print("Is prob dcp:",prob.is_dcp())
        result = prob.solve()
        print(self.a.value)
        self.W=np.zeros(f,dtype='float64')
        tempa=np.array(self.a.value)

        print(tempa)
        for i in range(m):
            if tempa[i]==self.C.value:
                self.W+=y_train[i]* tempa[i]* X[i]
        cnt_pos_vec=0
        # print("W:",type(W))
        self.W=np.asarray(self.W,dtype='float64')
        for i in range(m):
            if tempa[i]==self.C.value:
                # print(X[i].dtype)
                
                # print("y:",self.W.dtype)
                # print("h",y.dtype)
                # print((self.W* np.transpose(X[i])).shape)
                self.b+=y_train[i]-np.matmul(self.W, np.transpose(X[i]))
                cnt_pos_vec+=1
        y_train[y_train==-1]=0
        self.b/=max(1,cnt_pos_vec)
    def predict(self,X):
        # ycap=np.matmul(self.W,np.transpose(X))
        matb=np.full((X.shape[0],1),self.b)
        matw=np.full((X.shape[0],X.shape[1]),self.W)
        # print(matb)
        # print(matw)
        # print("shapE,",np.matmul(X,np.transpose(self.W)).shape)
        print(np.matmul(X,np.transpose(self.W)))
        # print("fnal:,",np.add(matb,np.matmul(X,np.transpose(self.W))).shape)
        y_cap=self.b+np.transpose(np.matmul(X,np.transpose(self.W)))
        print(y_cap.shape)
        y_cap[y_cap<=0]=0
        y_cap[y_cap>0]=1
        return y_cap
        
        
        

        
        
        



    
    
    
    

In [414]:
import pandas as pd
import numpy as np
from sklearn import svm
import math

In [415]:
##global variables
tz=0
tpr_by_fpr_min_thresh=0

tc=0
target_column_name='income'

In [417]:
data=pd.read_csv("datasets/adult.csv")

In [418]:
data['income'].replace(['<=50K', '>50K'],
                        [0, 1], inplace=True)

In [419]:
list_of_catagorical_vars=['workclass','education',
       'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country',
]


In [420]:
from sklearn.preprocessing import LabelEncoder
encoded_to_orig_dict={}
for col_name in list_of_catagorical_vars:
    le = LabelEncoder()
    le.fit(data[col_name])
    data[col_name]=le.transform(data[col_name])
    

    encoded_to_orig_dict[col_name]= dict((v,k) for k,v in dict(zip(le.classes_, le.transform(le.classes_))).items())

In [421]:
traindt=data;
test=data[target_column_name]
traindt.drop(target_column_name,axis=1,inplace=True)


In [422]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(traindt,test, test_size=0.25) # 70% training and 30% test

In [423]:
clf=SVMCust()

In [424]:
# X_train.shape
# y_test=y_test[:500]
# X_test=X_test[:500]
# X_train=X_train[:1000]
# y_train=y_train[:1000]

In [425]:
clf.fit(X_train, y_train)

shape: (24420, 14)


 10%|█         | 2472/24420 [09:45<1:27:19,  4.19it/s]

In [335]:
alphas=clf.getAlphas().value

In [336]:
alphas[alphas==0.02]=1
alphas[alphas<0.02]=0

In [337]:
y_train.value_counts()

0    750
1    250
Name: income, dtype: int64

In [338]:
y_pred=clf.predict(X_test)

35       3.593886e+10
11121    2.867784e+10
23138    2.434502e+10
26320    2.812044e+11
2118     3.132693e+11
             ...     
11922    1.420802e+11
27411    1.262587e+11
5267     4.075116e+11
22285    1.844288e+11
4195     1.804710e+11
Length: 500, dtype: float64
(500,)


In [339]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.528


In [340]:
from sklearn.metrics import confusion_matrix

In [341]:
confusion_matrix(y_test,y_pred)

array([[202, 177],
       [ 59,  62]], dtype=int64)

In [342]:
test_df_final=pd.concat([X_test,y_test],axis=1)

In [343]:
all_test=pd.concat([X_train,y_train],axis=1)

In [344]:
all_test['Pred']=alphas

In [345]:
true_predictions=all_test.loc[all_test[target_column_name]==all_test['Pred']]
true_positive_predictions=true_predictions.loc[true_predictions['Pred']==1]
# true_positive_predictions
true_negative_predictons=true_predictions.loc[true_predictions['Pred']==0]

In [346]:
all_test[target_column_name].value_counts()

0    750
1    250
Name: income, dtype: int64

In [347]:
true_predictions[target_column_name].value_counts()

0    750
1    250
Name: income, dtype: int64

In [348]:
true_positive_predictions.drop('Pred',axis=1,inplace=True)
true_negative_predictons.drop('Pred',axis=1,inplace=True)

c:\Users\acer\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [349]:
true_predictions_x=true_predictions.drop([target_column_name,'Pred'],axis=1)
true_predictions_y=true_predictions[target_column_name]

In [350]:
true_predictions_x= true_predictions_x.loc[:, (true_predictions_x != true_predictions_x.iloc[0]).any()] 

In [351]:
list_of_features_with_non_0_var=list(true_predictions_x.columns)
list_of_features_with_non_0_var

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [352]:
true_negative_predictons.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income'],
      dtype='object')

In [353]:
list_of_features_with_non_0_var

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [354]:
true_negative_predictons=true_negative_predictons[list_of_features_with_non_0_var+[target_column_name]]
true_positive_predictions=true_positive_predictions[list_of_features_with_non_0_var+[target_column_name]]

In [355]:

column_mean_for_STP_dict={}
column_mean_for_STN_dict={}
column_std_for_STP_dict={}
column_std_for_STN_dict={}
for column_name in list_of_features_with_non_0_var:
    if column_name not in list_of_catagorical_vars:
        print(column_name)
        column_mean_for_STP_dict[column_name]=true_positive_predictions[column_name].mean()
        column_mean_for_STN_dict[column_name]=true_negative_predictons[column_name].mean()
        column_std_for_STN_dict[column_name]=true_positive_predictions[column_name].std()
        column_std_for_STP_dict[column_name]=true_negative_predictons[column_name].std()

age
fnlwgt
education.num
capital.gain
capital.loss
hours.per.week


In [378]:
column_mean_for_STN_dict

{'age': 37.165333333333336,
 'fnlwgt': 191408.36133333333,
 'education.num': 9.588,
 'capital.gain': 138.364,
 'capital.loss': 33.436,
 'hours.per.week': 38.16133333333333}

In [380]:
column_mean_for_STP_dict

{'age': 43.536,
 'fnlwgt': 183438.192,
 'education.num': 11.82,
 'capital.gain': 6128.18,
 'capital.loss': 184.568,
 'hours.per.week': 44.928}

In [356]:
num_of_samples_in_STP=len(true_positive_predictions.columns)
num_of_samples_in_STN=len(true_negative_predictons.columns)

In [357]:
def getZmeanDiff(columnName):
    num=column_mean_for_STN_dict[column_name]+column_mean_for_STP_dict[column_name]
    dnum=((column_std_for_STN_dict[column_name]**2)/num_of_samples_in_STN)+((column_mean_for_STP_dict[column_name]**2)/num_of_samples_in_STP)
    dnum=math.sqrt(dnum)
    return num/dnum

In [358]:
from math import exp
from sklearn.utils import resample


def getChiSquaredTest(columnName):
    expected_dict={}
    obs_dict={}
    unique_val_in_stp=list(true_positive_predictions[column_name].unique())
    unique_val_in_stn=list(true_negative_predictons[col_name].unique())
    # print(unique_val_in_stp," ",unique_val_in_stn)
    obs_freq={}
    exp_freq={}
    for val in (unique_val_in_stn+ unique_val_in_stp) :
        obs_freq[val]=0
        exp_freq[val]=0
    # print(obs_freq)
    # print(exp_freq)
    tndict=pd.value_counts(true_negative_predictons[col_name]).to_dict()
    tpdict=pd.value_counts(true_positive_predictions[col_name]).to_dict()
    res=0
    # print("Res:",res)
    # print(obs_freq.keys())
    # print(exp_freq.keys())
    for val in (unique_val_in_stn+unique_val_in_stp):
        if val in tpdict.keys():
            obs_freq[val]=(tpdict[val]/ true_positive_predictions.shape[1])
        if val in tndict.keys():
            exp_freq[val]=(tndict[val]/ true_negative_predictons.shape[1])
        # print(obs_dic/t[val])
        # print(exp_)
        if exp_freq[val]==0:
            exp_freq[val]=0.2
        res+=((obs_freq[val]-exp_freq[val])**2)/exp_freq[val]
    return res
    
    
    


    

In [359]:
list_of_features_with_non_0_var

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [385]:
Z_diff_btw_two_means_dict={}
chi_squared_dict={}
for column_name in list_of_features_with_non_0_var:
    # print("cn:",column_name)
    if column_name not in list_of_catagorical_vars:
        Z_diff_btw_two_means_dict[column_name]=getZmeanDiff(column_name)
    else:
        chi_squared_dict[column_name]=getChiSquaredTest(column_name)

In [386]:
Z_diff_btw_two_means_dict

{'age': 6.982951333275138,
 'fnlwgt': 6.974757483652663,
 'education.num': 6.879413617465857,
 'capital.gain': 1.173060710139346,
 'capital.loss': 1.4075112240191132,
 'hours.per.week': 6.930263543725029}

In [387]:
chi_squared_dict

{'workclass': 24.958003992015957,
 'education': 25.030226214238176,
 'marital.status': 24.691337325349288,
 'occupation': 25.430226214238175,
 'relationship': 24.674670658682622,
 'race': 24.691337325349288,
 'sex': 24.341337325349294,
 'native.country': 43.70045242847637}

In [388]:
F=[]#
for column_name in list_of_features_with_non_0_var:
    if( column_name not in list_of_catagorical_vars and  Z_diff_btw_two_means_dict[column_name]>=tz):
        F+=[column_name]
    elif (column_name in list_of_catagorical_vars and chi_squared_dict[col_name]>=tc):
        F+=[column_name]

In [389]:
F

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education.num',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital.gain',
 'capital.loss',
 'hours.per.week',
 'native.country']

In [390]:
def get_tpr_to_fpr_ratio(df,column_name,thresh,target_column_name, iscategorical=0):
    tp=0
    fp=0
    fn=0
    tn=0
    if iscategorical==0:
        for index,row in df.iterrows():
            if(row[column_name]>=thresh):
                if(row[target_column_name]==1):
                    tp+=1
                else:
                    fp+=1
            else:
                if(row[target_column_name]==0):
                    tn+=1
                else:
                    fn+=1
    else:
        for index,row in df.iterrows():
            if(row[column_name]==thresh):
                if(row[target_column_name]==1):
                    tp+=1
                else:
                    fp+=1
            else:
                if(row[target_column_name]==0):
                    tn+=1
                else:
                    fn+=1

    tpr=tp/(fn+tp)
    fpr=fp/(fn+tp)
    if fpr==0:
        fpr=10000000
    # print("TPR:",tpr," Fpr:",fpr)
    return tpr/fpr

        
   
def get_aib_for_feature(column_name,df,step,target_column_name):
    myet=0
    aib=-1
    start_val=int(math.floor(df[column_name].min()))
    end_val=int(math.ceil(df[column_name].max()))
    step=max(1,int((end_val-start_val)/200))
    for poss_thres in range(start_val,end_val,step):
        val=get_tpr_to_fpr_ratio(df,column_name,poss_thres,target_column_name)
        # print("for thresh:",poss_thres," val:",val,"myet:", myet)
        if(myet<val):
            aib=poss_thres
            myet=val
    return aib





In [391]:
# def get_tpr_to_fpr_ratio(df,column_name,val,target_column_name):
def get_aib_for_cat_feature(column_name,df,target_column_name):
    local_dict={}
    for value in df[column_name].unique():
        local_dict[(column_name,value)]=get_tpr_to_fpr_ratio(df,column_name,value,target_column_name,iscategorical=1)
    return local_dict  
        
        


In [392]:
from tqdm import tqdm
# tempf=['Pregnancies']
aib_dict_for_F={}
aib_dict_for_cat_F={}
best_tpr_by_fpr_for_feature={}
for column_name in tqdm(F):
    if column_name not in list_of_catagorical_vars:
        aib_dict_for_F[column_name]=get_aib_for_feature(column_name,true_predictions,1,target_column_name)
        best_tpr_by_fpr_for_feature[column_name]=get_tpr_to_fpr_ratio(true_predictions,column_name,aib_dict_for_F[column_name],target_column_name)
    else :
        temp_dict=get_aib_for_cat_feature(column_name,true_predictions,target_column_name)
        aib_dict_for_cat_F.update(temp_dict)
        best_tpr_by_fpr_for_feature.update(temp_dict)
        

100%|██████████| 14/14 [00:43<00:00,  3.14s/it]


In [393]:
ranked_best_tpr_by_fpr_features=sorted(best_tpr_by_fpr_for_feature, key=lambda k: best_tpr_by_fpr_for_feature[k],reverse=True)

In [394]:
all_rule_list=[]
for value in ranked_best_tpr_by_fpr_features:
    
    if type(value) is not tuple:
        temp_dict={
            "column_name":value,
            "sign":">",
            "thresh":aib_dict_for_F[value]
        }
    else :
        temp_dict={
            "column_name":value[0],
            "sign":"==",
            "thresh":value[1]
        }

    all_rule_list.append(temp_dict)
all_rule_list

[{'column_name': 'capital.gain', 'sign': '>', 'thresh': 5489},
 {'column_name': 'education', 'sign': '==', 'thresh': 14},
 {'column_name': 'capital.loss', 'sign': '>', 'thresh': 1884},
 {'column_name': 'education.num', 'sign': '>', 'thresh': 15},
 {'column_name': 'education', 'sign': '==', 'thresh': 10},
 {'column_name': 'hours.per.week', 'sign': '>', 'thresh': 96},
 {'column_name': 'native.country', 'sign': '==', 'thresh': 9},
 {'column_name': 'education', 'sign': '==', 'thresh': 12},
 {'column_name': 'workclass', 'sign': '==', 'thresh': 5},
 {'column_name': 'native.country', 'sign': '==', 'thresh': 3},
 {'column_name': 'native.country', 'sign': '==', 'thresh': 17},
 {'column_name': 'relationship', 'sign': '==', 'thresh': 5},
 {'column_name': 'occupation', 'sign': '==', 'thresh': 10},
 {'column_name': 'occupation', 'sign': '==', 'thresh': 4},
 {'column_name': 'relationship', 'sign': '==', 'thresh': 0},
 {'column_name': 'marital.status', 'sign': '==', 'thresh': 2},
 {'column_name': 'wo

In [395]:
def isRuleSatisfied(rule_list,row):
    for rule in rule_list:
        cname=rule['column_name']
        if rule['sign']=='>':
            if row[cname]<=rule['thresh']:
                return False
        elif rule['sign']=='==':
            if row[cname]!=rule['thresh']:
                return False
    return True

In [396]:
def get_tpr_by_fpr_for_rule_list(rule_list,df,target_column_name):
    tp=0
    tn=0
    fp=0
    fn=0
    for index,row in df.iterrows():
        
        if(isRuleSatisfied(rule_list,row)):
            if(row[target_column_name]==1):
                tp+=1
            else:
                fp+=1
        else:
            if(row[target_column_name]==0):
                tn+=1
            else:
                fn+=1
    tpr=0
    fpr=0
    if tp+fp==0:
        tpr=tp/0.1
    else:
        tpr=tp/(fn+tp)
    if fn+tp==0:
        fpr=fp/0.1
    else:
        fpr=fp/(fn+tp)
    # print("tp:",tp," fp:",fp," tn:",tn," fn:",fn,"TPR:",tpr," Fpr:",fpr)
    if fpr==0:
        return (tpr/2000000,0)
   
    return (tpr/fpr,fpr)
    
    
    

In [397]:
final_rule_list=[]

In [398]:
# def add_rule_rec(df,all_rule_list,pres_ind,pres_rule_list,last_tpr_by_fpr_ratio,target_column_name):
def delete_rows_satisfying_rule(df,rule_list,target_column_name):
    list_of_ind_to_drop=[]
    ind=0
    for index,row in df.iterrows():
        # print("ind:",index)
    
        if(isRuleSatisfied(rule_list,row)):
            if(row[target_column_name]==1):
                list_of_ind_to_drop+=[ind]
        else:
            if(row[target_column_name]==0):
                list_of_ind_to_drop+=[ind]
        ind+=1
    df.drop(df.index[[list_of_ind_to_drop]],inplace=True)
    return df
    



In [399]:
result_rule_list=[]
df=true_predictions.copy()
for init_rule_ind in tqdm(range(0,len(all_rule_list))):
    init_rule_list=[all_rule_list[init_rule_ind]]
    (init_tpr_by_fpr,_)=get_tpr_by_fpr_for_rule_list(init_rule_list,df,target_column_name)
    # print("rule list:", init_rule_list)
    # print("tpr/fppr:",init_tpr_by_fpr)
    # print("df shape::",df.shape)
    new_rule_list=init_rule_list.copy()
    last_ratio=tpr_by_fpr_min_thresh
    vis=[0]*len(all_rule_list)
    vis[init_rule_ind]=1
    max_tpr_fpr_ratio=init_tpr_by_fpr
    max_tpr_fpr_ratio_ind=-1
    while get_tpr_by_fpr_for_rule_list(new_rule_list,df,target_column_name)[0]> last_ratio:
        init_rule_list=new_rule_list
        max_tpr_fpr_ratio=0
        max_tpr_fpr_ratio_ind=-1
        for new_rule_ind in range(len(all_rule_list)):
            if(vis[new_rule_ind]==1):
                continue
            temp_new_rule_list=new_rule_list.copy()
            temp_new_rule_list+=[all_rule_list[new_rule_ind]]
            (new_tpr_by_fpr,fpr_for_list)=get_tpr_by_fpr_for_rule_list(temp_new_rule_list,df,target_column_name)
            # print("for ind:",new_rule_ind, "new rat:",new_tpr_by_fpr," fpr_for_list:",fpr_for_list)
            if new_tpr_by_fpr>max_tpr_fpr_ratio and fpr_for_list!=0:
                max_tpr_fpr_ratio=new_tpr_by_fpr
                max_tpr_fpr_ratio_ind=new_rule_ind
        # print("new tpr/fpr",max_tpr_fpr_ratio)
        if max_tpr_fpr_ratio_ind==-1:
            break
        # if max_tpr_fpr_ratio>tpr_by_fpr_min_thresh:
        
        vis[max_tpr_fpr_ratio_ind]=1
        new_rule_list+=[all_rule_list[max_tpr_fpr_ratio_ind]]
        
    # print( "init rule list:",init_rule_list)
    result_rule_list.append(init_rule_list)
    #removing rows from df which are already classified
    # if max_tpr_fpr_ratio_ind==-1:
    #     continue
    df=delete_rows_satisfying_rule(df,new_rule_list,target_column_name)
    # print("df shape:",df.shape)
            
        



    



  0%|          | 0/92 [00:00<?, ?it/s]c:\Users\acer\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexes\base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
100%|██████████| 92/92 [00:44<00:00,  2.07it/s]


In [400]:
result_rule_list

[[{'column_name': 'capital.gain', 'sign': '>', 'thresh': 5489},
  {'column_name': 'age', 'sign': '>', 'thresh': 37},
  {'column_name': 'native.country', 'sign': '==', 'thresh': 39},
  {'column_name': 'sex', 'sign': '==', 'thresh': 1},
  {'column_name': 'workclass', 'sign': '==', 'thresh': 4},
  {'column_name': 'education', 'sign': '==', 'thresh': 11},
  {'column_name': 'occupation', 'sign': '==', 'thresh': 4},
  {'column_name': 'relationship', 'sign': '==', 'thresh': 1}],
 [{'column_name': 'education', 'sign': '==', 'thresh': 14}],
 [{'column_name': 'capital.loss', 'sign': '>', 'thresh': 1884}],
 [{'column_name': 'education.num', 'sign': '>', 'thresh': 15}],
 [{'column_name': 'education', 'sign': '==', 'thresh': 10}],
 [{'column_name': 'hours.per.week', 'sign': '>', 'thresh': 96}],
 [{'column_name': 'native.country', 'sign': '==', 'thresh': 9}],
 [{'column_name': 'education', 'sign': '==', 'thresh': 12}],
 [{'column_name': 'workclass', 'sign': '==', 'thresh': 5}],
 [{'column_name': 'na

In [401]:
def get_rule_list_predictions(rule_list,df):
    y_pred=[]
    for index,row in df.iterrows():
        for rule in rule_list:
            if isRuleSatisfied(rule,row):
                y_pred+=[1]
                break
        else:
            y_pred+=[0]
           
    return y_pred

                
    

In [402]:
from sklearn import metrics

In [403]:
true_predictions[target_column_name].value_counts()

0    750
1    250
Name: income, dtype: int64

In [404]:
post_pruned_rule_list=[]
last_auc_area=0
for rule in tqdm(result_rule_list):
    temp_y_pred=get_rule_list_predictions(post_pruned_rule_list+[rule],true_predictions)
    # print(temp_y_pred)
    # print(true_predictions)
    # print(" pred len:",len(temp_y_pred))
    fpr, tpr, thresholds = metrics.roc_curve(true_predictions[target_column_name],temp_y_pred)
    pres_auc=metrics.auc(fpr,tpr)
    print("pres auc",pres_auc)
    if(pres_auc-last_auc_area>=0.05):
        post_pruned_rule_list.append(rule)
        last_auc_area=pres_auc

  3%|▎         | 3/92 [00:00<00:03, 23.27it/s]

pres auc 0.5013333333333334
pres auc 0.5306666666666667
pres auc 0.538
pres auc 0.5293333333333333
pres auc 0.5293333333333333


 10%|▉         | 9/92 [00:00<00:04, 20.56it/s]

pres auc 0.5046666666666667
pres auc 0.5046666666666667
pres auc 0.546
pres auc 0.526


 13%|█▎        | 12/92 [00:00<00:03, 20.07it/s]

...

pres auc 0.7346666666666666
pres auc 0.7353333333333333
pres auc 0.7346666666666666
pres auc 0.7353333333333333


 95%|█████████▍| 87/92 [00:04<00:00, 17.72it/s]

pres auc 0.7346666666666666
pres auc 0.7353333333333333
pres auc 0.7346666666666666
pres auc 0.7346666666666666


100%|██████████| 92/92 [00:05<00:00, 17.43it/s]

pres auc 0.7346666666666666
pres auc 0.7346666666666666
pres auc 0.7346666666666666
pres auc 0.7353333333333333


In [405]:
post_pruned_rule_list

[[{'column_name': 'capital.gain', 'sign': '>', 'thresh': 5489},
  {'column_name': 'age', 'sign': '>', 'thresh': 37},
  {'column_name': 'native.country', 'sign': '==', 'thresh': 39},
  {'column_name': 'sex', 'sign': '==', 'thresh': 1},
  {'column_name': 'workclass', 'sign': '==', 'thresh': 4},
  {'column_name': 'education', 'sign': '==', 'thresh': 11},
  {'column_name': 'occupation', 'sign': '==', 'thresh': 4},
  {'column_name': 'relationship', 'sign': '==', 'thresh': 1}],
 [{'column_name': 'occupation', 'sign': '==', 'thresh': 10}],
 [{'column_name': 'occupation', 'sign': '==', 'thresh': 4}],
 [{'column_name': 'relationship', 'sign': '==', 'thresh': 0}]]

In [406]:
def check_accuracy_of_rules_for_dataframe(rule_list,df,target_column_name):
    fp=0
    fn=0
    tp=0
    tn=0
    for index,row in df.iterrows():
        for rule in rule_list:
            if isRuleSatisfied(rule,row):
                if row[target_column_name]==1:
                    tp+=1
                else:
                    fp+=1
                break
        else:
            if row[target_column_name]==0:
                tn+=1
            else:
                fn+=1
    return (tp,tn,fp,fn)

In [407]:
(tp,tn,fp,fn)=check_accuracy_of_rules_for_dataframe(post_pruned_rule_list,test_df_final,target_column_name)

In [409]:
print("Accuracy of the rules:",(tp+tn)/(tp+tn+fp+fn))

Accuracy of the rules: 0.642


In [410]:
def print_rule_list(rule_list):
    final_res="\n"
    for rules in rule_list:
        res="IF "
        # print("r:",rules[0])
        for i in range(0,len(rules)):
            if i>0:
                res+=" AND "
            res+=rules[i]['column_name']
            res+=" "
            res+=rules[i]['sign']
            res+=" "
            if rules[i]['column_name'] in list_of_catagorical_vars:
                val=rules[i]['thresh']
                cname=rules[i]['column_name']    
                res+=str(encoded_to_orig_dict[cname][val])
            else:
                res+=str(rules[i]['thresh'])
        res+=" THEN OUTCOME IS 1"
        # print("res:",res)
        final_res+="\n"
        final_res+=res
    print(final_res)        

In [411]:
print_rule_list(post_pruned_rule_list)




IF capital.gain > 5489 AND age > 37 AND native.country == United-States AND sex == Male AND workclass == Private AND education == HS-grad AND occupation == Exec-managerial AND relationship == Not-in-family THEN OUTCOME IS 1
IF occupation == Prof-specialty THEN OUTCOME IS 1
IF occupation == Exec-managerial THEN OUTCOME IS 1
IF relationship == Husband THEN OUTCOME IS 1
